In [2]:
import io
import keras
import tensorflow as tf
import os
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
#test_df=pd.read_csv('drive/My Drive/Colab Notebooks/dataset/test_outs.txt',delimiter=",",header=None)
# Dataset is now stored in a Pandas Dataframe

Using TensorFlow backend.


In [0]:
file = open('drive/My Drive/Colab Notebooks/dataset/vocab.txt', 'r')
# read all text
vocab = file.readlines()
vocabulary={}
for w in vocab:
  vocabulary[w.strip()]=""
# close the file
file.close()

In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:
docs_filtered=[]
for d in docs:
  # split into tokens by white space
  tokens = d.split()
  temp=""
  for t in tokens:
    if t in vocabulary:
      temp=temp+" "+t
  docs_filtered.append(temp)


In [0]:
# integer encode the documents
from keras.preprocessing.text import one_hot
vocab_size = len(vocabulary)+1

encoded_docs = [one_hot(d, vocab_size) for d in docs_filtered]

In [7]:
# pad documents to a max length of 4 words
max_length = 200
padded_docs = keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

91795


In [8]:
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y.shape)


(91795, 8)


In [9]:
emb_dim=32
emb_model = tf.keras.Sequential([ 
    tf.keras.layers.Input(shape=(max_length,)) ,                           
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.GRU(128,return_sequences=True,dropout=0.3),
    tf.keras.layers.GRU(64,return_sequences=True,dropout=0.3),
    tf.keras.layers.GRU(32,return_sequences=True),
    tf.keras.layers.GRU(16),
    tf.keras.layers.Dense(8, activation="softmax")
])

emb_model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['acc'])
print(emb_model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           8000032   
_________________________________________________________________
gru (GRU)                    (None, 200, 128)          62208     
_________________________________________________________________
gru_1 (GRU)                  (None, 200, 64)           37248     
_________________________________________________________________
gru_2 (GRU)                  (None, 200, 32)           9408      
_________________________________________________________________
gru_3 (GRU)                  (None, 16)                2400      
_________________________________________________________________
dense (Dense)                (None, 8)                 136       
Total params: 8,111,432
Trainable params: 8,111,432
Non-trainable params: 0
______________________________________________

In [10]:
emb_model.fit(padded_docs, dummy_y,epochs=30,batch_size=256, verbose=1,validation_split=0.1)


Epoch 1/30
323/323 [==============================] - 36s 113ms/step - loss: 1.7014 - acc: 0.3309 - val_loss: 1.0955 - val_acc: 0.6023
Epoch 2/30
323/323 [==============================] - 35s 109ms/step - loss: 0.8252 - acc: 0.7000 - val_loss: 0.7885 - val_acc: 0.6972
Epoch 3/30
323/323 [==============================] - 35s 108ms/step - loss: 0.5005 - acc: 0.8404 - val_loss: 0.3950 - val_acc: 0.8821
Epoch 4/30
323/323 [==============================] - 35s 108ms/step - loss: 0.2521 - acc: 0.9297 - val_loss: 0.3430 - val_acc: 0.8977
Epoch 5/30
323/323 [==============================] - 35s 107ms/step - loss: 0.1545 - acc: 0.9602 - val_loss: 0.3346 - val_acc: 0.9035
Epoch 6/30
323/323 [==============================] - 35s 108ms/step - loss: 0.1011 - acc: 0.9751 - val_loss: 0.3435 - val_acc: 0.9077
Epoch 7/30
323/323 [==============================] - 35s 108ms/step - loss: 0.0804 - acc: 0.9802 - val_loss: 0.3574 - val_acc: 0.9040
Epoch 8/30
323/323 [==============================] - 3

In [11]:
emb_model1 = tf.keras.Model(inputs=emb_model.inputs , outputs=emb_model.layers[3].output)
emb_X = emb_model1.predict(padded_docs)
print(emb_X.shape)
emb_X=np.reshape(emb_X, (emb_X.shape[0],1,-1))

(91795, 200, 32)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emb_X, dummy_y, test_size=0.18)

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(75271, 1, 6400)
(75271, 8)
(16524, 1, 6400)
(16524, 8)


In [14]:
model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.GRU(2048,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True),
    tf.keras.layers.GRU(1024,return_sequences=True,dropout=0.3,recurrent_dropout=0.3),
    tf.keras.layers.GRU(512,return_sequences=True,dropout=0.3,recurrent_dropout=0.3),
    tf.keras.layers.GRU(128,return_sequences=True),
    tf.keras.layers.GRU(32),
    tf.keras.layers.Dense(8, activation="softmax")
])

In [15]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, 1, 2048)           51916800  
_________________________________________________________________
gru_5 (GRU)                  (None, 1, 1024)           9443328   
_________________________________________________________________
gru_6 (GRU)                  (None, 1, 512)            2362368   
_________________________________________________________________
gru_7 (GRU)                  (None, 1, 128)            246528    
_________________________________________________________________
gru_8 (GRU)                  (None, 32)                15552     
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
Total params: 63,984,840
Trainable params: 63,984,840
Non-trainable params: 0
__________________________________________

In [16]:
history = model.fit(X_train, y_train, epochs=15,batch_size=256, shuffle=False, verbose=1,validation_split=0.1)
model.save('saved_model/text_classifier_rnn_word_embedding_epoch_20')

Epoch 1/15
265/265 [==============================] - 11s 43ms/step - loss: 0.1216 - acc: 0.9754 - val_loss: 0.0643 - val_acc: 0.9849
Epoch 2/15
265/265 [==============================] - 10s 37ms/step - loss: 0.0649 - acc: 0.9847 - val_loss: 0.0591 - val_acc: 0.9868
Epoch 3/15
265/265 [==============================] - 10s 37ms/step - loss: 0.0575 - acc: 0.9863 - val_loss: 0.0501 - val_acc: 0.9896
Epoch 4/15
265/265 [==============================] - 10s 37ms/step - loss: 0.0538 - acc: 0.9874 - val_loss: 0.0489 - val_acc: 0.9891
Epoch 5/15
265/265 [==============================] - 10s 37ms/step - loss: 0.0523 - acc: 0.9880 - val_loss: 0.0557 - val_acc: 0.9875
Epoch 6/15
265/265 [==============================] - 10s 37ms/step - loss: 0.0496 - acc: 0.9886 - val_loss: 0.0526 - val_acc: 0.9882
Epoch 7/15
265/265 [==============================] - 10s 37ms/step - loss: 0.0478 - acc: 0.9886 - val_loss: 0.0595 - val_acc: 0.9867
Epoch 8/15
265/265 [==============================] - 10s 36ms

In [0]:
!ls saved_model

text_classifier_rnn_word_embedding_epoch_15
text_classifier_rnn_word_embedding_epoch_20


In [0]:
#model.load_weights('./checkpoints/text_classifier_rnn_word_embedding')
new_model = tf.keras.models.load_model('saved_model/text_classifier_rnn_word_embedding_epoch_20')

In [17]:
results = model.evaluate(X_test, y_test)
print(results)

517/517 [==============================] - 4s 8ms/step - loss: 0.0458 - acc: 0.9901
[0.0457540787756443, 0.9901355504989624]


In [0]:
pred=new_model.predict(padded_docs)
print(type(pred))